# SILERO TTS

In [ ]:
#!mkdir src
#!touch src/test.txt

# !echo "Я приближался к месту моего назначения. " >> src/test.txt
# !echo "Вокруг меня, прижимаясь к самой дороге, зеленел лес, изредка уступая место полянам, поросшим желтой осокою. " >> src/test.txt
# !echo "Солнце садилось уже который час, все никак не могло сесть и висело низко над горизонтом. " >> src/test.txt
# !echo "Машина катилась по узкой дороге, засыпанной хрустящим гравием. " >> src/test.txt
# !echo "Крупные камни я пускал под колесо, и каждый раз в багажнике лязгали и громыхали пустые канистры." >> src/test.txt

In [1]:
!mkdir src
!unzip src.zip -d ./src

Archive:  src.zip
  inflating: ./src/part001 - Том 1..txt  
  inflating: ./src/part002 - Пролог..txt  
  inflating: ./src/part003 - Глава 1..txt  
  inflating: ./src/part004 - Глава 2..txt  
  inflating: ./src/part005 - Глава 3..txt  
  inflating: ./src/part006 - Глава 4..txt  
  inflating: ./src/part007 - Глава 5..txt  


In [2]:
!git clone https://github.com/fnx4/silero-tts silero

Cloning into 'silero'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 162 (delta 79), reused 133 (delta 50), pack-reused 0
Receiving objects: 100% (162/162), 67.65 KiB | 1.13 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [3]:
!ffmpeg -version | grep "ffmpeg version"
!nvidia-smi

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
Sun Apr 14 12:29:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.07             Driver Version: 537.34       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:01:00.0  On |                  Off |
| 47%   45C    P0              62W / 360W |   1199MiB / 24564MiB |      3%      Default |
|                                         |                      |        

In [4]:
!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI -b v2.2 rvc
!wget -O rvc/assets/rmvpe/rmvpe.pt https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt
!wget -O rvc/assets/hubert/hubert_base.pt https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt
!pip install virtualenv
!virtualenv rvc/venv
!rvc/venv/bin/python -m pip install -r rvc/requirements.txt --quiet

Cloning into 'rvc'...
remote: Enumerating objects: 4777, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 4777 (delta 47), reused 62 (delta 23), pack-reused 4678
Receiving objects: 100% (4777/4777), 14.32 MiB | 5.31 MiB/s, done.
Resolving deltas: 100% (2964/2964), done.
--2024-04-14 12:29:35--  https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt
Resolving huggingface.co (huggingface.co)... 108.138.51.8, 108.138.51.41, 108.138.51.26, ...
Connecting to huggingface.co (huggingface.co)|108.138.51.8|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/ef/97/ef977053f017cde1fc0f89ac7ef0b33172a3d8fb8a840bb24e78b1c0f35f1e72/6d62215f4306e3ca278246188607209f09af3dc77ed4232efdd069798c4ec193?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27rmvpe.pt%3B+filename%3D%22rmvpe.pt%22%3B&Expires=1713355674&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRp

In [5]:
!wget https://huggingface.co/fnx/RVCv2-RMVPE-ru-saya/resolve/main/ru-saya-1000.zip
!unzip ru-saya-1000.zip -d rvc/assets/weights
!mv rvc/assets/weights/ru-saya/* rvc/assets/weights

--2024-04-14 12:37:38--  https://huggingface.co/fnx/RVCv2-RMVPE-ru-saya/resolve/main/ru-saya-1000.zip
Resolving huggingface.co (huggingface.co)... 108.138.51.41, 108.138.51.26, 108.138.51.8, ...
Connecting to huggingface.co (huggingface.co)|108.138.51.41|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/6f/97/6f97c90b4497aa8edea081713b1d3163392d9536cc43374f32ffc6a4c6fe7625/181d635b7f4405318c072567f4cf9ff8076c814803ec87d82ea2af9bc1450c70?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27ru-saya-1000.zip%3B+filename%3D%22ru-saya-1000.zip%22%3B&response-content-type=application%2Fzip&Expires=1713357458&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMzM1NzQ1OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82Zi85Ny82Zjk3YzkwYjQ0OTdhYThlZGVhMDgxNzEzYjFkMzE2MzM5MmQ5NTM2Y2M0MzM3NGYzMmZmYzZhNGM2ZmU3NjI1LzE4MWQ2MzViN2Y0NDA1MzE4YzA3MjU2N2Y0Y2Y5ZmY4MDc2

In [6]:
!pip install -r silero/requirements.txt

In [7]:
#%run silero/silero_tts.py -t 16 -s xenia -d cuda -r 48000 --merge --rvc --rvc_model_pth ru-saya-1000.pth -i ./src -o silero/output_raw
%run silero/silero_tts.py -t 16 -s xenia -d cuda -r 48000 --merge --rvc --rvc_model_pth ru-saya-1000.pth --rvc_model_index trained_IVF3468_Flat_nprobe_1_ru-saya_v2.index -i ./src -o silero/output_raw


Namespace(input='./src', output='silero/output_raw', threads='16', speaker='xenia', device='cuda', rate='48000', merge=True, rvc=True, rvc_model_pth='ru-saya-1000.pth', rvc_model_index='trained_IVF3468_Flat_nprobe_1_ru-saya_v2.index', rvc_transpose='3')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



file: ./src/part001 - Том 1..txt ...


 CHAPTER:   0%|          | 0/1 [00:00<?, ?it/s]

    TEXT:   0%|          | 0/1 [00:00<?, ?it/s]

...

    TEXT:   0%|          | 0/1 [00:00<?, ?it/s]

SENTENCE:   0%|          | 0/87 [00:00<?, ?it/s]


Merging and encoding, please wait...


  0%|          | 0/7 [00:00<?, ?it/s]


Finished!


In [8]:
!zip -r output.zip silero/output_raw/_opus
!ls -l output.zip

updating: silero/output_raw/_opus/ (stored 0%)
  adding: silero/output_raw/_opus/part006_-_Глава_4__000000.opus (deflated 0%)
  adding: silero/output_raw/_opus/part007_-_Глава_5__000000.opus (deflated 0%)
  adding: silero/output_raw/_opus/part003_-_Глава_1__000000.opus (deflated 0%)
  adding: silero/output_raw/_opus/part001_-_Том_1__000000.opus (deflated 0%)
  adding: silero/output_raw/_opus/part002_-_Пролог__000000.opus (deflated 0%)
  adding: silero/output_raw/_opus/part005_-_Глава_3__000000.opus (deflated 0%)
  adding: silero/output_raw/_opus/part004_-_Глава_2__000000.opus (deflated 0%)
-rw-r--r-- 1 root root 8341291 Apr 14 12:39 output.zip
